In [1]:
# import uvicorn
import requests
import json
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import JsonOutputParser
from agents.ollama_agent import OllamaAgent
from utils.paper_utils import PaperUtils

In [2]:
path_to_pdf = r"F:\AI_ML_Notebooks\Research_Paper_Agent\cache\2025-11-02_00-49-27_0000.pdf"

In [3]:
pdf_loader = PyMuPDFLoader(path_to_pdf, mode='single')
doc = pdf_loader.load()
doc_context = json.loads(doc[0].json())['page_content']

C:\Users\tejas\AppData\Local\Temp\ipykernel_25052\1097576656.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  doc_context = json.loads(doc[0].json())['page_content']


In [4]:
# llm = OllamaAgent(model="gemma3:4b", host="http://127.0.0.1:11434", temperature=0.1, seed=41, num_ctx=61440, )
# llm.get_num_tokens(doc_context)

In [5]:
# MCP server URL
BASE_URL = "http://127.0.0.1:8080"

# uvicorn.run("main:app", host="0.0.0.0", port=8080)  
url = f"{BASE_URL}/get_num_tokens"
params = {"query_context": doc_context}
response = requests.post(url, params=params)
print(f"num tokens: {response.json()}")


# url = f"{BASE_URL}/analyze_with_llm"    
# 🔍 Example query
query = "Summarize the document"
params = {"query": query, "query_context": doc_context}
url = f"{BASE_URL}/summarize_with_llm"
response = requests.post(url, params=params)
llm_response = response.json()['llm_response']
print(f"llm_response : {llm_response}")

num tokens: 11345
llm_response : ```json
{
  "summary": "Veo-3 demonstrates emerging visual reasoning capabilities, particularly in short-horizon spatial coherence and fine-grained grounding, but struggles with complex reasoning conditions like long-horizon causal consistency, strict geometric constraints, and abstract logic. While it shows encouraging signs as a complementary visual engine alongside specialized reasoning models, it is not yet reliable as a standalone zero-shot reasoner.",
  "overview": "The study comprehensively investigates Veo-3's reasoning potential across 12 dimensions, revealing strengths in short-horizon spatial coherence and local dynamics, but significant limitations in handling complex geometric constraints, causal reasoning, and abstract logic. The research highlights the model's potential as a visual engine alongside specialized reasoning models, emphasizing the need for further advancements in its reasoning capabilities."
}
```


In [6]:
json_parser = JsonOutputParser()
llm_response_json = json_parser.parse(llm_response)
llm_response_json

{'summary': 'Veo-3 demonstrates emerging visual reasoning capabilities, particularly in short-horizon spatial coherence and fine-grained grounding, but struggles with complex reasoning conditions like long-horizon causal consistency, strict geometric constraints, and abstract logic. While it shows encouraging signs as a complementary visual engine alongside specialized reasoning models, it is not yet reliable as a standalone zero-shot reasoner.',
 'overview': "The study comprehensively investigates Veo-3's reasoning potential across 12 dimensions, revealing strengths in short-horizon spatial coherence and local dynamics, but significant limitations in handling complex geometric constraints, causal reasoning, and abstract logic. The research highlights the model's potential as a visual engine alongside specialized reasoning models, emphasizing the need for further advancements in its reasoning capabilities."}

In [7]:
doc_json = json.loads(doc[0].json())
doc_json

C:\Users\tejas\AppData\Local\Temp\ipykernel_25052\4243559565.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  doc_json = json.loads(doc[0].json())


{'id': None,
 'metadata': {'producer': 'pikepdf 8.15.1',
  'creator': 'arXiv GenPDF (tex2pdf:e76afa9)',
  'creationdate': '',
  'source': 'F:\\AI_ML_Notebooks\\Research_Paper_Agent\\cache\\2025-11-02_00-49-27_0000.pdf',
  'file_path': 'F:\\AI_ML_Notebooks\\Research_Paper_Agent\\cache\\2025-11-02_00-49-27_0000.pdf',
  'total_pages': 36,
  'format': 'PDF 1.7',
  'title': 'Are Video Models Ready as Zero-Shot Reasoners? An Empirical Study with the MME-CoF Benchmark',
  'author': 'Ziyu Guo; Xinyan Chen; Renrui Zhang; Ruichuan An; Yu Qi; Dongzhi Jiang; Xiangtai Li; Manyuan Zhang; Hongsheng Li; Pheng-Ann Heng',
  'subject': '',
  'keywords': '',
  'moddate': '',
  'trapped': '',
  'modDate': '',
  'creationDate': ''},
 'page_content': 'Are Video Models Ready as Zero-Shot Reasoners?\nAn Empirical Study with the MME-COF Benchmark\nZiyu Guo∗†1, Xinyan Chen∗2, Renrui Zhang∗‡2, Ruichuan An∗3, Yu Qi∗4, Dongzhi Jiang2\nXiangtai Li3, Manyuan Zhang2, Hongsheng Li2, Pheng-Ann Heng1\nCUHK 1IMIXR & 2MMLa

In [8]:
final_answer = {
    'Title' : doc_json['metadata']['title'],
    'Summary' : llm_response_json['summary'],
    'Overview' : llm_response_json['overview']
}
final_answer    

{'Title': 'Are Video Models Ready as Zero-Shot Reasoners? An Empirical Study with the MME-CoF Benchmark',
 'Summary': 'Veo-3 demonstrates emerging visual reasoning capabilities, particularly in short-horizon spatial coherence and fine-grained grounding, but struggles with complex reasoning conditions like long-horizon causal consistency, strict geometric constraints, and abstract logic. While it shows encouraging signs as a complementary visual engine alongside specialized reasoning models, it is not yet reliable as a standalone zero-shot reasoner.',
 'Overview': "The study comprehensively investigates Veo-3's reasoning potential across 12 dimensions, revealing strengths in short-horizon spatial coherence and local dynamics, but significant limitations in handling complex geometric constraints, causal reasoning, and abstract logic. The research highlights the model's potential as a visual engine alongside specialized reasoning models, emphasizing the need for further advancements in it

In [9]:
url = f"{BASE_URL}/fetch_all_sources"
response = requests.get(url)
contexts = response.json()

In [10]:
contexts['arxiv'][0]

{'title': 'Agent-Omni: Test-Time Multimodal Reasoning via Model Coordination for\n  Understanding Anything',
 'summary': 'Multimodal large language models (MLLMs) have shown strong capabilities but\nremain limited to fixed modality pairs and require costly fine-tuning with\nlarge aligned datasets. Building fully omni-capable models that can integrate\ntext, images, audio, and video remains impractical and lacks robust reasoning\nsupport. In this paper, we propose an Agent-Omni framework that coordinates\nexisting foundation models through a master-agent system, enabling flexible\nmultimodal reasoning without retraining. The master agent interprets user\nintent, delegates subtasks to modality-specific agents, and integrates their\noutputs into coherent responses. Extensive experiments across text, image,\naudio, video, and omni benchmarks show that Agent-Omni consistently achieves\nstate-of-the-art performance, particularly on tasks requiring complex\ncross-modal reasoning. Its agent-ba

In [11]:
for item in contexts['arxiv']:
    pdf_doc = PaperUtils().process_paper(item)
    params = {"query": query, "query_context": pdf_doc['content']}
    url = f"{BASE_URL}/summarize_with_llm"
    response = requests.post(url, params=params)
    llm_response = response.json()['llm_response']
    llm_response_json = json_parser.parse(llm_response)
    final_answer = {
    'Title' : pdf_doc['title'],
    'Summary' : llm_response_json['summary'],
    'Overview' : llm_response_json['overview']
    }
    print(final_answer)

Saving pdf to: cache/Agent-OmniTest-Time Multimodal Reasoning via Model Coordination for Understanding Anything.pdf.
{'Title': 'Agent-OmniTest-Time Multimodal Reasoning via Model Coordination for Understanding Anything', 'Summary': 'The Agent-Omni framework proposes a novel approach to achieving truly omni-capable multimodal reasoning at test time. It coordinates existing foundation models through a master-agent system, enabling flexible integration of text, images, audio, and video inputs without retraining. The system interprets user intent, delegates subtasks to modality-specific agents, and integrates their outputs into coherent responses, consistently achieving state-of-the-art performance across diverse benchmarks.', 'Overview': 'Agent-Omni utilizes a hierarchical agent architecture. A master agent interprets user queries, identifies relevant modalities, and delegates subtasks to specialized foundation models (text, image, audio, video). These models generate structured outputs, 